In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5_normalize/recording9_fbank.h5' #訓練好的模型
tflite_filename = './tflite_normalize/recording9_fbank.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型


INFO:tensorflow:Assets written to: C:\Users\r6017\AppData\Local\Temp\tmp9z58r423\assets


1483968

In [4]:
#即時
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
import librosa
#import RPi.GPIO as GPIO

# Parameters
word_threshold = 0.7 #預測值>0.7
rec_duration = 0.5 #每一段錄音持續時間
sample_rate = 16000 #取樣率(依MIC不同而改變)
num_channels = 1 #音訊深度
model_path = './tflite_normalize/recording9_fbank.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '其他', '吼', '啦', '嗯', '好', '我', '的', '著', '那', '阿']

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)

    #將音訊輸入到window
    window = rec
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) >= 500: #判斷S的總和是否>3000，如果>3000，代表有講話
        
        window = window.astype(np.float)

        window = (window - window.mean()) / (window.max() - window.min())
    
        # Compute features
        features = python_speech_features.base.logfbank(window,
                                                        samplerate=16000,
                                                        winlen=0.025,
                                                        winstep=0.01,
                                                        nfilt=26,
                                                        nfft=512,
                                                        lowfreq=0,
                                                        highfreq=None,
                                                        preemph=0.97)
        
        # Make prediction from model
        in_tensor = np.float32(features.reshape(1, features.shape[0], features.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max > word_threshold):
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
    
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        
        pass


ㄟ
MAX:0.9979702830314636
Error: input overflow
backgroundNoise
MAX:0.9287272095680237
Error: input overflow
backgroundNoise
MAX:0.9996408224105835
Error: input overflow
backgroundNoise
MAX:0.9985847473144531
backgroundNoise
MAX:0.894828736782074
Error: input overflow
backgroundNoise
MAX:0.9999380111694336
backgroundNoise
MAX:0.9999487400054932
Error: input overflow
他
MAX:0.982633650302887
Error: input overflow
Error: input overflow
ㄟ
MAX:0.828911304473877
Error: input overflow
其他
MAX:0.9939947724342346
Error: input overflow
backgroundNoise
MAX:0.9999778270721436
Error: input overflow
backgroundNoise
MAX:0.9868867993354797
backgroundNoise
MAX:0.9978873133659363
backgroundNoise
MAX:0.7390699982643127
ㄟ
MAX:1.0
backgroundNoise
MAX:0.9040457606315613
backgroundNoise
MAX:0.9996330738067627
Error: input overflow
backgroundNoise
MAX:0.9999988079071045
backgroundNoise
MAX:0.8209375143051147
backgroundNoise
MAX:0.9998724460601807
backgroundNoise
MAX:0.9999916553497314
Error: input overflow
back

KeyboardInterrupt: 

In [11]:
#非即時
def sound(window,s,m):

    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) >= 500: #判斷S的總和是否>=500，如果>=500，代表有明顯的聲音
        window = window.astype(np.float)

        window = (window - window.mean()) / (window.max() - window.min())

        features = python_speech_features.base.logfbank(window,
                                                        samplerate=16000,
                                                        winlen=0.025,
                                                        winstep=0.01,
                                                        nfilt=26,
                                                        nfft=512,
                                                        lowfreq=0,
                                                        highfreq=None,
                                                        preemph=0.97)
        
        # Make prediction from model
        in_tensor = np.float32(features.reshape(1, features.shape[0], features.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max > 0.7):
            print(words[list_val_maxIndex] + " " + str(m) + "分" + str(s) + "秒" + " 預測值:" + str(list_val_max))#輸出相對應的字詞
        #if(best_candidate_probability > 0.7): # treshold
        #    if(str(all_targets[best_candidate_index]) != "backgroundNoise"):
        #        data.append(str(all_targets[best_candidate_index]) + " " + str(m) + "分" + str(s) + "秒" + " 預測值:" + str(best_candidate_probability)) 
        

#main
# Parameters
model_path = './tflite_normalize/recording9_fbank.tflite'
all_targets = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '其他', '吼', '啦', '嗯', '好', '我', '的', '著', '那', '阿']

data = []
start = 0 #一開始的索引值
end = 4000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 60 #讀音檔的總時間
sample_rate = 16000 #取樣率

#載入音檔
y, sr = librosa.load("./TestWAV/那.wav",sr=sample_rate) 

#用0填滿時間，以讓滑動視窗都有東西
total_sec = len(y) / 16000 #算音檔總時間
if total_sec < 1: #如果音檔<1s
    new_y = np.zeros(16000) #new_y補足1s
    new_y[0:len(y)] = y #將原先的音訊載入到新的陣列
elif total_sec - (int(total_sec)) != 0: #如果音檔非整數秒，ex:原先音訊長度1.2s ，1.2 - 1 != 0 ， 將長度補到2s
    total_sec = (int(total_sec) + 1) #新時間為原時間整數+1
    new_y = np.zeros(total_sec * 16000) #補足到整數秒
    new_y[0:len(y)] = y #將原先的音訊載入到新的陣列
else:
    new_y = y

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

while True:
    s = s + 0.25 #增加秒數
    if(s >= 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    window[:4000] = window[4000:] #把音訊載入window
    window[4000:] = new_y[start:end] #把音訊載入window
    
    sound(window,s,m) #呼叫sound()

    if(end == len(new_y)): #如果移動到最後，break
        break
    
    start = start + 4000 #向後移動
    end = end + 4000 #向後移動    

backgroundNoise 0分0.25秒 預測值:0.9999947547912598
那 0分0.5秒 預測值:0.9993242025375366
那 0分0.75秒 預測值:0.9999938011169434
著 0分1.25秒 預測值:1.0
那 0分1.5秒 預測值:1.0
那 0分1.75秒 預測值:0.9999667406082153
著 0分2.25秒 預測值:0.9962903261184692
那 0分2.5秒 預測值:1.0
那 0分2.75秒 預測值:0.999151349067688
backgroundNoise 0分3.25秒 預測值:0.8335757851600647
那 0分3.5秒 預測值:0.9999856948852539
那 0分3.75秒 預測值:0.9997172951698303
backgroundNoise 0分4.25秒 預測值:0.9999486207962036
那 0分4.5秒 預測值:0.9967143535614014
那 0分4.75秒 預測值:0.9994006156921387
那 0分5.0秒 預測值:0.7175742387771606
吼 0分5.25秒 預測值:0.9967585206031799
那 0分5.75秒 預測值:1.0
的 0分6.0秒 預測值:0.7779529094696045
著 0分6.5秒 預測值:0.8818113803863525
那 0分6.75秒 預測值:1.0
著 0分7.5秒 預測值:0.9999362230300903
那 0分7.75秒 預測值:1.0
那 0分8.0秒 預測值:0.7278148531913757
著 0分8.5秒 預測值:0.7013989090919495
那 0分8.75秒 預測值:0.9966236352920532
吼 0分9.5秒 預測值:0.9985641837120056
著 0分9.75秒 預測值:0.9102464914321899
那 0分10.0秒 預測值:0.9785610437393188
其他 0分10.25秒 預測值:0.9817657470703125
